In [1]:
import pandas as pd 

data = pd.read_csv('Data/full_rnn_data_2.csv')
data.drop('Unnamed: 0', axis = 1, inplace= True)
data

C:\Users\domen\AppData\Local\Temp\ipykernel_9300\2725669544.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Data/full_rnn_data_2.csv')


,lastTradeDate,strike,price,bid,ask,change,percentChange,volume,openInterest,sigma,...,ticker_ZS,ticker_ZTO,ticker_ZTS,ticker_ZUMZ,ticker_ZUO,ticker_ZVIA,ticker_ZVRA,ticker_ZWS,ticker_ZYME,ticker_ZYXI
0,2024-07-23,70.0,0.18,137.05,137.12,0.0,0.0,1717668.0,2.0,0.263417,...,False,False,False,False,False,False,False,False,False,False
1,2024-07-23,70.0,0.18,137.05,137.12,0.0,0.0,1717668.0,2.0,0.263417,...,False,False,False,False,False,False,False,False,False,False
2,2024-05-02,85.0,0.10,137.05,137.12,0.0,0.0,1717668.0,16.0,0.263417,...,False,False,False,False,False,False,False,False,False,False
3,2024-05-02,85.0,0.10,137.05,137.12,0.0,0.0,1717668.0,16.0,0.263417,...,False,False,False,False,False,False,False,False,False,False
4,2024-05-02,85.0,0.10,137.05,137.12,0.0,0.0,1717668.0,16.0,0.263417,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278709,2024-05-15,17.5,0.35,8.90,8.96,0.0,0.0,1034596.0,109.0,0.599988,...,False,False,False,False,False,False,False,False,False,True
278710,2024-03-06,20.0,0.50,8.90,8.96,0.0,0.0,1034596.0,2.0,0.599988,...,False,False,False,False,False,False,False,False,False,True
278711,2024-03-06,20.0,0.50,8.90,8.96,0.0,0.0,1034596.0,2.0,0.599988,...,False,False,False,False,False,False,False,False,False,True
278712,2024-03-06,20.0,0.50,8.90,8.96,0.0,0.0,1034596.0,2.0,0.599988,...,False,False,False,False,False,False,False,False,False,True


In [2]:
data = data.replace({False: 0, True: 1})

In [3]:
data.drop('lastTradeDate', axis = 1, inplace= True)

In [4]:
ann3_data = data

ann3_call_data = ann3_data[ann3_data.call == 1]
ann3_put_data = ann3_data[ann3_data.call == 0]

ann3_call_data.drop('call', axis = 1, inplace = True)
ann3_put_data.drop('call', axis = 1, inplace = True)

C:\Users\domen\AppData\Local\Temp\ipykernel_9300\43202908.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ann3_call_data.drop('call', axis = 1, inplace = True)
C:\Users\domen\AppData\Local\Temp\ipykernel_9300\43202908.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ann3_put_data.drop('call', axis = 1, inplace = True)


In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler

# Load your data
# ann3_call_data = pd.read_csv('your_data.csv')  # Uncomment and modify this line to load your data

# Separate features and target
y = ann3_call_data['price']
X = ann3_call_data.drop('price', axis=1)

# Handle categorical data by one-hot encoding
X = pd.get_dummies(X, drop_first=True)

# Replace infinite values with NaN
X.replace([np.inf, -np.inf], np.nan, inplace=True)

# Remove rows with NaN values
X.dropna(inplace=True)
y = y[X.index]  # Keep the target values corresponding to the filtered features

# Convert to more memory-efficient data types
X = X.astype(np.float32)
y = y.astype(np.float32)

# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Feature selection (optional, adjust k based on your needs)
k = 1000  # Number of top features to keep
selector = SelectKBest(f_regression, k=k)
X_selected = selector.fit_transform(X_scaled, y)

print(f"Data shape after feature selection: {X_selected.shape}")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.20, random_state=0)

# Reshape the data to 3D format for LSTM (samples, timesteps, features)
X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Build RNN model
RNN_model = Sequential()
RNN_model.add(LSTM(400, input_shape=(1, X_train_reshaped.shape[2]), activation='relu', return_sequences=True))
RNN_model.add(LSTM(400, activation='relu', return_sequences=True))
RNN_model.add(LSTM(400, activation='relu'))
RNN_model.add(Dense(1))

# Compile the model with a lower learning rate and gradient clipping
optimizer = Adam(learning_rate=0.001, clipnorm=1.0)
RNN_model.compile(loss='mean_squared_error', optimizer=optimizer)

# Train the RNN model
batch_size = 512  # Adjust based on memory constraints
RNN_model.fit(X_train_reshaped, y_train, epochs=40, batch_size=16, verbose=1)

# Evaluate the model
loss = RNN_model.evaluate(X_test_reshaped, y_test)
print(f"Test Loss: {loss}")


Data shape after feature selection: (149092, 1000)

Epoch 1/40

7455/7455 [==============================] - 506s 67ms/step - loss: 820.9005
Epoch 2/40
7455/7455 [==============================] - 524s 70ms/step - loss: 575.1749
Epoch 3/40
7455/7455 [==============================] - 525s 70ms/step - loss: 410.0237
Epoch 4/40
7455/7455 [==============================] - 528s 71ms/step - loss: 277.6516
Epoch 5/40
7455/7455 [==============================] - 526s 71ms/step - loss: 221.6249
Epoch 6/40
7455/7455 [==============================] - 529s 71ms/step - loss: 197.2964
Epoch 7/40
7455/7455 [==============================] - 531s 71ms/step - loss: 154.7865
Epoch 8/40
7455/7455 [==============================] - 535s 72ms/step - loss: 146.3054
Epoch 9/40
7455/7455 [==============================] - 534s 72ms/step - loss: 139.6914
Epoch 10/40
7455/7455 [==============================] - 536s 72ms/step - loss: 119.0134
Epoch 11/40
7455/7455 [==============================] - 535s 72m

In [6]:
RNN_model.save('models\\rnn\RNN_model.keras')

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler

# Load your data
# ann3_call_data = pd.read_csv('your_data.csv')  # Uncomment and modify this line to load your data

# Separate features and target
y = ann3_put_data['price']
X = ann3_put_data.drop('price', axis=1)

# Handle categorical data by one-hot encoding
X = pd.get_dummies(X, drop_first=True)

# Replace infinite values with NaN
X.replace([np.inf, -np.inf], np.nan, inplace=True)

# Remove rows with NaN values
X.dropna(inplace=True)
y = y[X.index]  # Keep the target values corresponding to the filtered features

# Convert to more memory-efficient data types
X = X.astype(np.float32)
y = y.astype(np.float32)

# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Feature selection (optional, adjust k based on your needs)
k = 1000  # Number of top features to keep
selector = SelectKBest(f_regression, k=k)
X_selected = selector.fit_transform(X_scaled, y)

print(f"Data shape after feature selection: {X_selected.shape}")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.20, random_state=0)

# Reshape the data to 3D format for LSTM (samples, timesteps, features)
X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Build RNN model
RNN_put = Sequential()
RNN_put.add(LSTM(400, input_shape=(1, X_train_reshaped.shape[2]), activation='relu', return_sequences=True))
RNN_put.add(LSTM(400, activation='relu', return_sequences=True))
RNN_put.add(LSTM(400, activation='relu'))
RNN_put.add(Dense(1))

# Compile the model with a lower learning rate and gradient clipping
optimizer = Adam(learning_rate=0.001, clipnorm=1.0)
RNN_put.compile(loss='mean_squared_error', optimizer=optimizer)

# Train the RNN model
batch_size = 512  # Adjust based on memory constraints
RNN_put.fit(X_train_reshaped, y_train, epochs=40, batch_size=16, verbose=1)

# Evaluate the model
loss = RNN_put.evaluate(X_test_reshaped, y_test)
print(f"Test Loss: {loss}")


Data shape after feature selection: (128952, 1000)
Epoch 1/40
6448/6448 [==============================] - 429s 66ms/step - loss: 513.4741
Epoch 2/40
6448/6448 [==============================] - 428s 66ms/step - loss: 400.8830
Epoch 3/40
6448/6448 [==============================] - 425s 66ms/step - loss: 302.5637
Epoch 4/40
6448/6448 [==============================] - 425s 66ms/step - loss: 230.2968
Epoch 5/40
6448/6448 [==============================] - 425s 66ms/step - loss: 169.3755
Epoch 6/40
6448/6448 [==============================] - 436s 68ms/step - loss: 142.6531
Epoch 7/40
6448/6448 [==============================] - 467s 72ms/step - loss: 132.5991
Epoch 8/40
6448/6448 [==============================] - 466s 72ms/step - loss: 114.2160
Epoch 9/40
6448/6448 [==============================] - 463s 72ms/step - loss: 102.9977
Epoch 10/40
6448/6448 [==============================] - 463s 72ms/step - loss: 96.9478
Epoch 11/40
6448/6448 [==============================] - 462s 72ms/st

KeyboardInterrupt: 

In [8]:
RNN_put.save('models\\rnn\RNN_put.keras')